<div style="width: 50%; float: left">
    <img src="../ups2.png" width="217" align="center">
    <br>
    <strong>CARRERA: </strong> Computación
    <br>
    <strong>NRO. PRÁCTICA: </strong>4
    <br>
    <strong>ESTUDIANTE: </strong>Doménica Merchán García
    <br>
</div>
<div style="width: 50%; float: left">
    <br>
    <strong align="center">PRÁCTICA DE LABORATORIO</strong>
    <br>
    <br>
    <br>
    <strong>ASIGNATURA: </strong>Simulación
    <br>
    <strong>TÍTULO: </strong>Simulación del proceso de vacunación en Ecuador
    <br>
    <br>
    <br>
</div>
<strong><center>ACTIVIDADES DESARROLLADAS</center></strong>

In [3]:
import simpy
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Se toma uno de los recintos en la parroquia El Batán de Cuenca. En esta zona hay 12738 personas que deberán votar en uno de los tres Recintos Electorales [1]. Suponiendo que todas las personas se reparten equitativamente en los 3 recintos, en cada recinto se atenderán a 4246 personas. 

Puesto a que sólo el 80% de la población será vacunada, de las 4246 personas que deberían asistir al recinto, se espera que realmente asistan 3397 personas.

Puesto a que el horario de atención para la vacunación es de 08h00 a 17h00 [2], se define el tiempo de ejecución de la simulación diario en 540 (9 horas en minutos)

In [5]:
TIEMPO = 540
PUNTOS_VACUNACION0 = 6
PUNTOS_VACUNACION = 6
PUNTOS_VACUNACION2 = int(PUNTOS_VACUNACION/2)
DIAS = 100

vacunados1 = []
no_vacunados1 = []

s1 = 0
n1 = 0

vacunados2 = []
no_vacunados2 = []

s2 = 0
n2 = 0

tiempo_espera = []
tiempo = 0

Se crea la clase Centro Covid donde se simularán todos los procesos a llevarse a cabo durante la vacunación de las personas del recinto.

In [6]:
class Centro_Covid(object):
    def __init__(self, env, puntos_vacunacion):
        self.env = env
        self.puntos_vacunacion = simpy.Resource(env, puntos_vacunacion)
        
    def poner_vacuna(self, paciente, dosis):
        global s1
        global s2
        global tiempo
        print('> El %s va a vacunarse a las %.2f. %s' % (paciente, env.now, dosis))
        t = random.randint(5,10)
        tiempo += t
        yield self.env.timeout(t)
        print('> El %s ha sido vacunado a las %.2f. %s' % (paciente, env.now, dosis))
        if '1' in dosis:
            s1+=1
        else:
            s2+=1
    
    def esperar(self, paciente, dosis):
        global tiempo
        print('_  El %s comienza su tiempo de espera a las %.2f. %s' % (paciente, env.now, dosis))
        tiempo += 20
        yield self.env.timeout(20)
        print('_  El %s termina su tiempo de espera a las %.2f. %s' % (paciente, env.now, dosis))
        yield self.env.process(self.certificar(paciente, dosis))
        
    def certificar(self, paciente, dosis):
        global tiempo
        print('@   El %s va a recibir su certificado a las %.2f. %s' % (paciente, env.now, dosis))
        t = random.randint(2,3)
        tiempo += t
        yield self.env.timeout(t)
        print('@   El %s ha recibido su certificado a las %.2f. %s y sale del recinto' % (paciente, env.now, dosis))

Se crea también un método para simular la llegada del paciente al recinto para comenzar su proceso de vacunación.

In [7]:
def llega_paciente(env, paciente, centro_covid, dosis):
    global n1
    global n2
    print('El %s llega al centro de vacunación a las %.2f. %s' % (paciente, env.now, dosis))
    with centro_covid.puntos_vacunacion.request() as punto_vacunacion:
        if random.randint(1,10) < 9:
            yield punto_vacunacion
            yield env.process(centro_covid.poner_vacuna(paciente, dosis))
            env.process(centro_covid.esperar(paciente, dosis))
        else:
            print('! El %s no ha podido vacunarse la %s y sale del recinto a las %.2f.' % (paciente, dosis, env.now))
            if '1' in dosis:
                n1+=1
            else:
                n2+=1

Se inicia la simulación con la llegada del paciente al recinto, realizando la reserva de los recursos y la asignación de tiempos de ejecución.

In [8]:
def ejecutar(env, puntos_vacunacion, dosis):
    centro_covid = Centro_Covid(env, puntos_vacunacion)
    for i in range(PUNTOS_VACUNACION):
        env.process(llega_paciente(env, 'Paciente %d' %(i+1), centro_covid, dosis))
        
    while True:
        yield env.timeout(random.randint(3,15))
        i+=1
        env.process(llega_paciente(env, 'Paciente %d' %(i+1), centro_covid, dosis))

In [9]:
random.seed(88)

for i in range(DIAS):
    print()
    print('------ DÍA ' + str(i+1) + ' ------')
    print()
    
    s1 = 0
    n1 = 0
    s2 = 0
    n2 = 0
    tiempo = 0
    
    env = simpy.Environment()
    if i >= 30:
        if sum(vacunados2) < 3397:
            env.process(ejecutar(env, PUNTOS_VACUNACION, '2da dosis'))
        if sum(vacunados1) < 3397:
            PUNTOS_VACUNACION = PUNTOS_VACUNACION2
        else:
            PUNTOS_VACUNACION = PUNTOS_VACUNACION0
    
    if sum(vacunados1) < 3397:
        env.process(ejecutar(env, PUNTOS_VACUNACION, '1ra dosis'))

        
    env.run(until=TIEMPO)
    
    vacunados1.append(s1)
    no_vacunados1.append(n1)
    vacunados2.append(s2)
    no_vacunados2.append(n2)
    tiempo_espera.append(tiempo)


------ DÍA 1 ------

El Paciente 1 llega al centro de vacunación a las 0.00. 1ra dosis
El Paciente 2 llega al centro de vacunación a las 0.00. 1ra dosis
El Paciente 3 llega al centro de vacunación a las 0.00. 1ra dosis
El Paciente 4 llega al centro de vacunación a las 0.00. 1ra dosis
El Paciente 5 llega al centro de vacunación a las 0.00. 1ra dosis
! El Paciente 5 no ha podido vacunarse la 1ra dosis y sale del recinto a las 0.00.
El Paciente 6 llega al centro de vacunación a las 0.00. 1ra dosis
> El Paciente 1 va a vacunarse a las 0.00. 1ra dosis
> El Paciente 2 va a vacunarse a las 0.00. 1ra dosis
> El Paciente 3 va a vacunarse a las 0.00. 1ra dosis
> El Paciente 4 va a vacunarse a las 0.00. 1ra dosis
> El Paciente 6 va a vacunarse a las 0.00. 1ra dosis
> El Paciente 4 ha sido vacunado a las 6.00. 1ra dosis
> El Paciente 6 ha sido vacunado a las 6.00. 1ra dosis
_  El Paciente 4 comienza su tiempo de espera a las 6.00. 1ra dosis
_  El Paciente 6 comienza su tiempo de espera a las 6.00

El Paciente 27 llega al centro de vacunación a las 199.00. 1ra dosis
> El Paciente 27 va a vacunarse a las 199.00. 1ra dosis
@   El Paciente 24 ha recibido su certificado a las 201.00. 1ra dosis y sale del recinto
> El Paciente 26 ha sido vacunado a las 204.00. 1ra dosis
El Paciente 28 llega al centro de vacunación a las 204.00. 1ra dosis
_  El Paciente 26 comienza su tiempo de espera a las 204.00. 1ra dosis
> El Paciente 28 va a vacunarse a las 204.00. 1ra dosis
> El Paciente 27 ha sido vacunado a las 207.00. 1ra dosis
_  El Paciente 27 comienza su tiempo de espera a las 207.00. 1ra dosis
> El Paciente 28 ha sido vacunado a las 211.00. 1ra dosis
_  El Paciente 28 comienza su tiempo de espera a las 211.00. 1ra dosis
_  El Paciente 25 termina su tiempo de espera a las 213.00. 1ra dosis
@   El Paciente 25 va a recibir su certificado a las 213.00. 1ra dosis
El Paciente 29 llega al centro de vacunación a las 213.00. 1ra dosis
! El Paciente 29 no ha podido vacunarse la 1ra dosis y sale del 

_  El Paciente 8 comienza su tiempo de espera a las 30.00. 1ra dosis
@   El Paciente 2 ha recibido su certificado a las 32.00. 1ra dosis y sale del recinto
@   El Paciente 6 ha recibido su certificado a las 32.00. 1ra dosis y sale del recinto
> El Paciente 9 ha sido vacunado a las 33.00. 1ra dosis
@   El Paciente 5 ha recibido su certificado a las 33.00. 1ra dosis y sale del recinto
_  El Paciente 9 comienza su tiempo de espera a las 33.00. 1ra dosis
El Paciente 10 llega al centro de vacunación a las 35.00. 1ra dosis
> El Paciente 10 va a vacunarse a las 35.00. 1ra dosis
_  El Paciente 7 termina su tiempo de espera a las 39.00. 1ra dosis
@   El Paciente 7 va a recibir su certificado a las 39.00. 1ra dosis
El Paciente 11 llega al centro de vacunación a las 41.00. 1ra dosis
> El Paciente 10 ha sido vacunado a las 41.00. 1ra dosis
@   El Paciente 7 ha recibido su certificado a las 41.00. 1ra dosis y sale del recinto
> El Paciente 11 va a vacunarse a las 41.00. 1ra dosis
_  El Paciente 10 

El Paciente 31 llega al centro de vacunación a las 241.00. 1ra dosis
> El Paciente 31 va a vacunarse a las 241.00. 1ra dosis
@   El Paciente 28 ha recibido su certificado a las 242.00. 1ra dosis y sale del recinto
@   El Paciente 29 ha recibido su certificado a las 244.00. 1ra dosis y sale del recinto
El Paciente 32 llega al centro de vacunación a las 249.00. 1ra dosis
> El Paciente 32 va a vacunarse a las 249.00. 1ra dosis
> El Paciente 31 ha sido vacunado a las 250.00. 1ra dosis
_  El Paciente 31 comienza su tiempo de espera a las 250.00. 1ra dosis
El Paciente 33 llega al centro de vacunación a las 253.00. 1ra dosis
> El Paciente 33 va a vacunarse a las 253.00. 1ra dosis
_  El Paciente 30 termina su tiempo de espera a las 254.00. 1ra dosis
@   El Paciente 30 va a recibir su certificado a las 254.00. 1ra dosis
@   El Paciente 30 ha recibido su certificado a las 256.00. 1ra dosis y sale del recinto
> El Paciente 32 ha sido vacunado a las 258.00. 1ra dosis
_  El Paciente 32 comienza su 

! El Paciente 45 no ha podido vacunarse la 1ra dosis y sale del recinto a las 338.00.
El Paciente 46 llega al centro de vacunación a las 341.00. 1ra dosis
> El Paciente 46 va a vacunarse a las 341.00. 1ra dosis
> El Paciente 44 ha sido vacunado a las 343.00. 1ra dosis
_  El Paciente 44 comienza su tiempo de espera a las 343.00. 1ra dosis
_  El Paciente 41 termina su tiempo de espera a las 350.00. 1ra dosis
@   El Paciente 41 va a recibir su certificado a las 350.00. 1ra dosis
> El Paciente 46 ha sido vacunado a las 350.00. 1ra dosis
_  El Paciente 46 comienza su tiempo de espera a las 350.00. 1ra dosis
El Paciente 47 llega al centro de vacunación a las 352.00. 1ra dosis
! El Paciente 47 no ha podido vacunarse la 1ra dosis y sale del recinto a las 352.00.
@   El Paciente 41 ha recibido su certificado a las 352.00. 1ra dosis y sale del recinto
El Paciente 48 llega al centro de vacunación a las 355.00. 1ra dosis
> El Paciente 48 va a vacunarse a las 355.00. 1ra dosis
_  El Paciente 43 ter

...

> El Paciente 14 va a vacunarse a las 76.00. 2da dosis
@   El Paciente 12 ha recibido su certificado a las 78.00. 2da dosis y sale del recinto
> El Paciente 14 ha sido vacunado a las 84.00. 2da dosis
_  El Paciente 14 comienza su tiempo de espera a las 84.00. 2da dosis
El Paciente 15 llega al centro de vacunación a las 91.00. 2da dosis
> El Paciente 15 va a vacunarse a las 91.00. 2da dosis
El Paciente 16 llega al centro de vacunación a las 95.00. 2da dosis
> El Paciente 16 va a vacunarse a las 95.00. 2da dosis
> El Paciente 15 ha sido vacunado a las 99.00. 2da dosis
_  El Paciente 15 comienza su tiempo de espera a las 99.00. 2da dosis
El Paciente 17 llega al centro de vacunación a las 103.00. 2da dosis
> El Paciente 17 va a vacunarse a las 103.00. 2da dosis
_  El Paciente 14 termina su tiempo de espera a las 104.00. 2da dosis
@   El Paciente 14 va a recibir su certificado a las 104.00. 2da dosis
> El Paciente 16 ha sido vacunado a las 105.00. 2da dosis
_  El Paciente 16 comienza su tie

_  El Paciente 41 comienza su tiempo de espera a las 311.00. 2da dosis
_  El Paciente 42 comienza su tiempo de espera a las 311.00. 2da dosis
El Paciente 43 llega al centro de vacunación a las 314.00. 2da dosis
! El Paciente 43 no ha podido vacunarse la 2da dosis y sale del recinto a las 314.00.
_  El Paciente 40 termina su tiempo de espera a las 317.00. 2da dosis
@   El Paciente 40 va a recibir su certificado a las 317.00. 2da dosis
@   El Paciente 40 ha recibido su certificado a las 320.00. 2da dosis y sale del recinto
El Paciente 44 llega al centro de vacunación a las 327.00. 2da dosis
> El Paciente 44 va a vacunarse a las 327.00. 2da dosis
_  El Paciente 41 termina su tiempo de espera a las 331.00. 2da dosis
@   El Paciente 41 va a recibir su certificado a las 331.00. 2da dosis
_  El Paciente 42 termina su tiempo de espera a las 331.00. 2da dosis
@   El Paciente 42 va a recibir su certificado a las 331.00. 2da dosis
@   El Paciente 41 ha recibido su certificado a las 333.00. 2da do

El Paciente 18 llega al centro de vacunación a las 90.00. 2da dosis
> El Paciente 18 va a vacunarse a las 90.00. 2da dosis
> El Paciente 17 ha sido vacunado a las 91.00. 2da dosis
_  El Paciente 17 comienza su tiempo de espera a las 91.00. 2da dosis
_  El Paciente 15 termina su tiempo de espera a las 96.00. 2da dosis
@   El Paciente 15 va a recibir su certificado a las 96.00. 2da dosis
> El Paciente 18 ha sido vacunado a las 96.00. 2da dosis
_  El Paciente 18 comienza su tiempo de espera a las 96.00. 2da dosis
_  El Paciente 16 termina su tiempo de espera a las 97.00. 2da dosis
@   El Paciente 16 va a recibir su certificado a las 97.00. 2da dosis
@   El Paciente 15 ha recibido su certificado a las 98.00. 2da dosis y sale del recinto
@   El Paciente 16 ha recibido su certificado a las 99.00. 2da dosis y sale del recinto
El Paciente 19 llega al centro de vacunación a las 104.00. 2da dosis
> El Paciente 19 va a vacunarse a las 104.00. 2da dosis
El Paciente 20 llega al centro de vacunación

> El Paciente 52 ha sido vacunado a las 390.00. 2da dosis
_  El Paciente 52 comienza su tiempo de espera a las 390.00. 2da dosis
@   El Paciente 49 ha recibido su certificado a las 391.00. 2da dosis y sale del recinto
El Paciente 54 llega al centro de vacunación a las 399.00. 2da dosis
> El Paciente 54 va a vacunarse a las 399.00. 2da dosis
_  El Paciente 51 termina su tiempo de espera a las 401.00. 2da dosis
@   El Paciente 51 va a recibir su certificado a las 401.00. 2da dosis
@   El Paciente 51 ha recibido su certificado a las 404.00. 2da dosis y sale del recinto
El Paciente 55 llega al centro de vacunación a las 409.00. 2da dosis
> El Paciente 54 ha sido vacunado a las 409.00. 2da dosis
> El Paciente 55 va a vacunarse a las 409.00. 2da dosis
_  El Paciente 54 comienza su tiempo de espera a las 409.00. 2da dosis
_  El Paciente 52 termina su tiempo de espera a las 410.00. 2da dosis
@   El Paciente 52 va a recibir su certificado a las 410.00. 2da dosis
@   El Paciente 52 ha recibido s

_  El Paciente 26 comienza su tiempo de espera a las 175.00. 2da dosis
El Paciente 27 llega al centro de vacunación a las 177.00. 2da dosis
> El Paciente 27 va a vacunarse a las 177.00. 2da dosis
_  El Paciente 25 termina su tiempo de espera a las 185.00. 2da dosis
@   El Paciente 25 va a recibir su certificado a las 185.00. 2da dosis
El Paciente 28 llega al centro de vacunación a las 185.00. 2da dosis
> El Paciente 27 ha sido vacunado a las 185.00. 2da dosis
> El Paciente 28 va a vacunarse a las 185.00. 2da dosis
_  El Paciente 27 comienza su tiempo de espera a las 185.00. 2da dosis
@   El Paciente 25 ha recibido su certificado a las 188.00. 2da dosis y sale del recinto
El Paciente 29 llega al centro de vacunación a las 189.00. 2da dosis
> El Paciente 29 va a vacunarse a las 189.00. 2da dosis
> El Paciente 28 ha sido vacunado a las 193.00. 2da dosis
_  El Paciente 28 comienza su tiempo de espera a las 193.00. 2da dosis
> El Paciente 29 ha sido vacunado a las 194.00. 2da dosis
_  El Pa

In [14]:
print('Cantidad de personas vacunadas con la primera dosis: ', sum(vacunados1))
print('Cantidad de personas no vacunadas durante la primera dosis: ', sum(no_vacunados1))
print()
print('Cantidad de personas vacunadas con la segunda dosis: ', sum(vacunados2))
print('Cantidad de personas no vacunadas durante la segunda dosis: ', sum(no_vacunados2))
print()
print('Tiempo de espera promedio', sum(tiempo_espera)/len(tiempo_espera))

Cantidad de personas vacunadas con la primera dosis:  3402
Cantidad de personas no vacunadas durante la primera dosis:  849

Cantidad de personas vacunadas con la segunda dosis:  3405
Cantidad de personas no vacunadas durante la segunda dosis:  826

Tiempo de espera promedio 2046.11
